# Finding the highest weight path

## Artificial example

In [ ]:
import bw_processing as bwp
import matrix_utils as mu
import sknetwork as skn
import numpy as np
from scipy import sparse
from typing import List

In [ ]:
A = 101
B = 102
C = 103
D = 104

In [ ]:
edges = np.array([
    (A, A),
    (B, B),
    (C, C),
    (D, D),
    (A, B),
    (A, C),
    (A, D),
    (B, C),
    (C, D),
    (D, C),
], dtype=bwp.INDICES_DTYPE)
data = np.array([1, 2, 1, 1, 2, 0.5, 0.01, 3, 1, 0.2])
flip = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=bool)

In [ ]:
dp = bwp.create_datapackage()
dp.add_persistent_vector(
    matrix="example",
    data_array = data,
    indices_array=edges,
    flip_array=flip
)

In [ ]:
mm = mu.MappedMatrix(packages=[dp], matrix="example")

In [ ]:
def to_normalized_adjacency_matrix(mapped_matrix: mu.MappedMatrix, log_transform: bool = True) -> sparse.csr_matrix:
    """Take a technosphere matrix constructed with Brightway conventions, and return a normalized adjacency matrix.
    
    In the adjacency matrix A, `A[i,j]` indicates a directed edge **from** row `i` **to** column `j`. This fits 
    the Brightway mode, where `A[i,j]` means **activity** `j` consumes the output of activity `i`. In other words,
    this matrix is ready for traversal from a functional unit to a supplier; to go in the opposite direction,
    use the transpose of this matrix.
    
    Normalization is done to remove the effect of activities which don't produce one unit of their reference product. 
    For example, if activity `foo` produces two units of `bar` and consumes two units of `baz`, the weight of the 
    `baz` edge should be 2 / 2 = 1.
    
    In addition to this normalization, we subtract the diagonal and flip the signs of all matrix values. Flipping 
    the sign is needed because we want to use a shortest path algorithm, but actually want the longest path. The
    longest path is the path with the highest weight, i.e. the path where the most consumption occurs on.
    
    By default, we also take the natural log of the data values. This is because our supply chain is multiplicative,
    not additive, and $ a \cdot b = e^{\ln(a) + \ln(b)}$.
    
    Assumes that production amounts are on the diagonal.
    """
    matrix = mapped_matrix.matrix.tocsr()

    # TBD these values should the NET production values, i.e. we use the production exchanges to get the matrix
    # indices, ensure that we have 1-1 match for production-activity, construct the normalization vector, turn
    # into a diagonal matrix, and then multiply
    normalization = sparse.diags(-1 * matrix.diagonal())
    normalized = ((matrix * normalization) + sparse.eye(*matrix.shape))

    if log_transform:
        normalized = normalized.tocoo()
        normalized.data = np.log(normalized.data) * -1
        normalized = normalized.tocsr()
        
    return normalized

In [ ]:
skn.path.get_shortest_path(
    adjacency = to_normalized_adjacency_matrix(mapped_matrix = mm),
    sources = 0,
    targets = 3,
    method = 'BF', 
    unweighted = False,
)

## With ecoinvent

In [ ]:
import bw2data as bd
import bw2calc as bc

In [ ]:
bd.projects.set_current("ecoinvent-3.9-cutoff")

In [ ]:
wafer = bd.get_activity(name="market for wafer, fabricated, for integrated circuit")

In [ ]:
sg = bd.get_activity(name='sugarcane production', location="BR-PR")

In [ ]:
def path_as_brightway_objects(source_node: bd.Node, target_node: bd.Node) -> List[bd.Edge]:
    lca = bc.LCA({source_node: 1, target_node: 1})
    lca.lci()
    
    path = skn.path.get_shortest_path(
        adjacency = to_normalized_adjacency_matrix(mapped_matrix = lca.technosphere_mm),
        sources = lca.activity_dict[source_node.id],
        targets = lca.activity_dict[target_node.id],
        method = 'BF', 
        unweighted = False,
    )
    
    return [
        (
            bd.get_node(id=lca.dicts.product.reversed[x]),
            bd.get_node(id=lca.dicts.activity.reversed[y]),
            -1 * lca.technosphere_matrix[x,y],
        )
        for x, y in zip(path[:-1], path[1:])
    ]

## With ecoinvent and uncertain matrix samples

In [ ]:
source_node, target_node = wafer, sg

lca = bc.LCA({source_node: 1, target_node: 1}, use_distributions=True)
lca.lci()

path = skn.path.get_shortest_path(
    adjacency = to_normalized_adjacency_matrix(mapped_matrix = lca.technosphere_mm),
    sources = lca.activity_dict[source_node.id],
    targets = lca.activity_dict[target_node.id],
    method = 'BF', 
    unweighted = False,
)

[
    (
        bd.get_node(id=lca.dicts.product.reversed[x]),
        bd.get_node(id=lca.dicts.activity.reversed[y]),
        -1 * lca.technosphere_matrix[x,y],
    )
    for x, y in zip(path[:-1], path[1:])
]